In [1]:
import cv2
import numpy as np
from matplotlib import pyplot as plt
import pandas as pd
import random
from PIL import Image
import os
import csv
import sklearn.metrics as skm

### 取得原始影像資料

1. 將影像轉成灰階
2. 轉換成二維像素陣列

In [2]:
def getImageArray(dirName):
    imageArray=[]
    for imageName in os.listdir(dirName):
        imageArray.append(cv2.imread(dirName+'/'+imageName))
    return imageArray,len(imageArray)

In [3]:
def getResizeImageArray():
    imageResizeArray=[]
    imageArray,imageArrayLength=getImageArray('../database/resizeImage')
    for image in imageArray:
        imageResizeArray.append(cv2.cvtColor(image,cv2.COLOR_BGR2GRAY))
    return imageResizeArray,imageArrayLength

In [4]:
imageArray,imageArrayLength=getResizeImageArray()
testImage=imageArray[0]

### 取得資訊

1. 取得加密訊息
2. 轉為ascii碼
3. 轉換為二進制序列(0bxxxxxxx)
4. 去除0b
5. 向右對齊補0
6. 存入變數

In [5]:
def getMessage(txtPath):
    byteNumber=0
    with open(txtPath,"rb")  as file:
        txt = file.read()
        byteNumber=len(txt)
        charMessage=""
        for index in range(len(txt)):
            charMessage=charMessage+""+str(bin(txt[index]).replace('0b','')).zfill(7)
            if(index<10):
                print('第'+str(index)+'位為:  '+chr(txt[index])+', 二進制為: '+str(bin(txt[index]).replace('0b','')).zfill(7))
            else:
                print('第'+str(index)+'位為: '+chr(txt[index])+', 二進制為: '+str(bin(txt[index]).replace('0b','')).zfill(7))
    print("輸出為："+charMessage) 
    print("字元數："+str(byteNumber)) 
    print("位元數："+str(len(charMessage))) 
    return charMessage

In [6]:
message=getMessage("../message/1024message.txt")

第0位為:  N, 二進制為: 1001110
第1位為:  I, 二進制為: 1001001
第2位為:  U, 二進制為: 1010101
第3位為:  1, 二進制為: 0110001
第4位為:  1, 二進制為: 0110001
第5位為:  1, 二進制為: 0110001
第6位為:  2, 二進制為: 0110010
第7位為:  C, 二進制為: 1000011
第8位為:  r, 二進制為: 1110010
第9位為:  y, 二進制為: 1111001
第10位為: p, 二進制為: 1110000
第11位為: t, 二進制為: 1110100
第12位為: o, 二進制為: 1101111
第13位為: g, 二進制為: 1100111
第14位為: r, 二進制為: 1110010
第15位為: a, 二進制為: 1100001
第16位為: p, 二進制為: 1110000
第17位為: h, 二進制為: 1101000
第18位為: y, 二進制為: 1111001
第19位為: N, 二進制為: 1001110
第20位為: I, 二進制為: 1001001
第21位為: U, 二進制為: 1010101
第22位為: 1, 二進制為: 0110001
第23位為: 1, 二進制為: 0110001
第24位為: 1, 二進制為: 0110001
第25位為: 2, 二進制為: 0110010
第26位為: C, 二進制為: 1000011
第27位為: r, 二進制為: 1110010
第28位為: y, 二進制為: 1111001
第29位為: p, 二進制為: 1110000
第30位為: t, 二進制為: 1110100
第31位為: o, 二進制為: 1101111
第32位為: g, 二進制為: 1100111
第33位為: r, 二進制為: 1110010
第34位為: a, 二進制為: 1100001
第35位為: p, 二進制為: 1110000
第36位為: h, 二進制為: 1101000
第37位為: y, 二進制為: 1111001
第38位為: N, 二進制為: 1001110
第39位為: I, 二進制為: 1001001
第40位為: U, 二進制為: 1010101
第41位為: 1, 二進制為: 

In [7]:
#設定共享密鑰δ
key=3
keyIndex=key-1

### 通用函數

#### 取得資訊

getACharMessage(message,index) 取得單一字元二元序列

#### 區塊切割

1. showBlock(block) 打印7*7區塊
2. makeBlock(startx,starty,image) 將原始圖像切分成7*7大小的區塊
3. pixelToBinary(pixel) 像素轉成二維陣列
4. turnBlocksToBinary(block) 像素區塊轉二元序列

#### LSB提取

binaryToLSB(binary) 取得LSB

#### 奇同位修改 

1. XOR(a,b,c) 三位元XOR運算
2. oddParityCheckRow(row) 一列的奇同位修改
3. coverBlock(LSBBlock) 生成掩護圖像

#### 找尋嵌入訊息位置

1. getShareSecretPosition(rowIndex,lastSecretIndex,embeddedIndex) 共享秘密位置 κi 
2. chooseEmbeddeedIndex(row,secretPosition,message) 選擇嵌入位置
3. embeddLSBBlock(LSBBlock,message) 嵌入訊息到LSB區塊

#### 區塊復原

1. reviseBinaryBlock(index,binaryPixelBlocks,LSBBlocks) LSB區塊修改二元序列區塊的最後一個位元
2. revisePixelBlock(index,binaryPixelBlocks,pixelBlocks) 二元序列區塊還原成十進制像素區塊

#### 區塊復原到像素

putPixelBlockBack(startx,starty,pixelBlock,hidedImage) 將區塊復原到隱藏訊息影像陣列

#### 儲存影藏訊息陣列成圖像

storeImage(hidedImage,path) 儲存隱藏訊息圖像

#### PSNR

psnr(originImage,hidedImage) 計算 psnr

#### 相對熵 

hxx_forward(x, y) 計算相對熵

In [8]:
# 打印7*7區塊
def showBlock(block):
    for row in block:
        for element in row:
            print(element,end=' ')
        print()
    print('\n-----------------------------------------\n')

In [9]:
# 將原始圖像切分成7*7大小的區塊
def makeBlock(startx,starty,image):
    block=[]
    for row in range(7):
        block.append(image[startx+row][starty:starty+7])
    # showBlock(block)
    return block

In [10]:
# 像素轉成二維陣列
def pixelToBinary(pixel):
    return str(bin(pixel).replace('0b','').zfill(8))

In [11]:
# 像素區塊轉二元序列
def turnBlocksToBinary(block):
    binaryPixelBlock=[]
    for row in block:
        binaryRow=[]
        for pixel in row:
            binaryRow.append(pixelToBinary(pixel))
        binaryPixelBlock.append(binaryRow)

    # showBlock(binaryPixelBlock)
    return binaryPixelBlock

In [12]:
# 取得LSB
def binaryToLSB(binary):
    return int(binary[-1])

In [13]:
# 三位元XOR運算
def XOR(a,b,c):
    string=str(a)+str(b)+str(c)
    switcher = {
        '000': 0,
        '001': 1,
        '010': 1,
        '011': 0,
        '100': 1,
        '101': 0,
        '110': 0,
        '111': 1
    }
    return switcher.get(string, 0)

In [14]:
# 將二元序列區塊轉換LSB區塊
def turnBinaryBlockToLSBBlock(binaryBlock):
    LSBPixelBlock=[]
    for row in binaryBlock:
        LSBRow=[]
        for binaryPixel in row:
            LSBRow.append(binaryToLSB(binaryPixel))
        LSBPixelBlock.append(LSBRow)
    # showBlock(LSBPixelBlock)
    return LSBPixelBlock

In [15]:
# 一列的奇同位修改
def oddParityCheckRow(row):
    opcRow=row
    if(int(XOR(row[2],row[4],row[6]))==1):
        opcRow[0]=0
    if(int(XOR(row[2],row[5],row[6]))==0):
        opcRow[1]=1
    if(int(XOR(row[4],row[5],row[6]))==0):
        row[3]=1
    return opcRow

In [16]:
# 生成掩護圖像
def coverBlock(LSBBlock):
    # print('原始區塊: ')
    # showBlock(LSBBlock)
    for index,LSBRow in enumerate(LSBBlock):
        LSBBlock[index]=oddParityCheckRow(LSBRow)
    # print('奇同位修改區塊: ')
    # showBlock(LSBBlock)

In [17]:
#共享秘密位置 κi 
def getShareSecretPosition(rowIndex,lastSecretIndex,embeddedIndex):
    if(rowIndex==0):
        return int(key%7)
    else:
        return int((key*lastSecretIndex*embeddedIndex)%7)
    return 0

In [18]:
# 選擇嵌入位置
def chooseEmbeddeedIndex(row,secretPosition,message):
    indexPossible=[i for i in range(7)]
    
    #去除共享秘密位置 κi
    indexPossible.remove(secretPosition)
    
    for index,value in enumerate(row):
        if(index==secretPosition):
            continue
        if(value==message):
            indexPossible.remove(index)

    if(len(indexPossible)==0):
        return secretPosition

    return random.choice(indexPossible)

In [19]:
# 嵌入訊息到LSB區塊
def embeddLSBBlock(LSBBlock,message):
    lastSecretIndex=0
    embeddedIndex=0
    for index,row in enumerate(LSBBlock):
        lastSecretIndex=getShareSecretPosition(index,lastSecretIndex,embeddedIndex)
        #翻轉位元
        row[lastSecretIndex]=abs(int(row[lastSecretIndex])-1)
        embeddedIndex=chooseEmbeddeedIndex(row,lastSecretIndex,int(message[index]))
        row[embeddedIndex]=message[index]

In [20]:
#取得單一字元二元序列
def getACharMessage(message,index):
    return message[(index*7):(index*7)+7]

In [21]:
#LSB區塊修改二元序列區塊的最後一個位元
def reviseBinaryBlock(index,binaryPixelBlocks,LSBBlocks):
    binaryBlock=binaryPixelBlocks[index]
    LSBBlock=LSBBlocks[index]
    for blockRowIndex in range(7):
        binaryBlockRow=binaryBlock[blockRowIndex]
        LSBBlockRow=LSBBlock[blockRowIndex]
        for elementIndex in range(7):
            binaryBlockRow[elementIndex]=binaryBlockRow[elementIndex][:-1]+str(LSBBlockRow[elementIndex])
        binaryBlock[blockRowIndex]=binaryBlockRow
    binaryPixelBlocks[index]=binaryBlock

In [22]:
# 二元序列區塊還原成十進制像素區塊
def revisePixelBlock(index,binaryPixelBlocks,pixelBlocks):
    binaryBlock=binaryPixelBlocks[index]
    pixelBlock=pixelBlocks[index]
    for blockRowIndex in range(7):
        binaryBlockRow=binaryBlock[blockRowIndex]
        BlockRow=pixelBlock[blockRowIndex]
        for elementIndex in range(7):
            BlockRow[elementIndex]=int(binaryBlockRow[elementIndex],2)   
        pixelBlock[blockRowIndex]=BlockRow
    pixelBlocks[index]=pixelBlock

In [23]:
# 將區塊復原到隱藏訊息影像陣列
def putPixelBlockBack(startx,starty,pixelBlock,hidedImage):
    for i in range(7):
        for j in range(7):
            hidedImage[starty*7+j][startx*7+i]=pixelBlock[j][i]

In [24]:
# 儲存隱藏訊息圖像
def storeImage(hidedImage,path):
    hidedImage=pd.DataFrame(hidedImage)
    cv2.imwrite(path,hidedImage.to_numpy())

In [25]:
# 計算 psnr
def psnr(originImage,hidedImage):
   diffrence = originImage-hidedImage
   mse = np.mean(np.square(diffrence))
   psnr = 10 * np.log10(255 * 255 / mse)
   return(psnr)

In [26]:
#計算相對熵
def hxx_forward(x, y):
    return skm.mutual_info_score(x, y)

def hxx(x, y):
    size = x.shape[-1]
    px = np.histogram(x, 256, (0, 255))[0] / size
    py = np.histogram(y, 256, (0, 255))[0] / size
    hx = - np.sum(px * np.log(px + 1e-8))
    hy = - np.sum(py * np.log(py + 1e-8))
 
    hxy = np.histogram2d(x, y, 256, [[0, 255], [0, 255]])[0]
    hxy /= (1.0 * size)
    hxy = - np.sum(hxy * np.log(hxy + 1e-8))
 
    r = hx + hy - hxy
    return r

### 單一影像生成

#### 輸入

 testImage: 單一圖像

 message: 隱藏訊息

 prt: 是否顯示提示
 
 path: 隱藏圖像儲存路徑

In [34]:
def PRDHHCSingle(testImage,message,prt,path):
    pixelBlocks=[]
    binaryPixelBlocks=[]
    LSBBlocks=[]
    blockWidth=int(testImage.shape[0]/7)
    blockHeight=int(testImage.shape[1]/7)
    blocksSize=blockWidth*blockHeight

    #切分原始區塊
    if(prt==True):
        print('原圖像素: '+str(testImage.shape))
        print("區塊數量: "+str(blockWidth)+' * '+str(blockHeight))
        print("總區塊數量: "+str(blocksSize))
    
    for startx in range(blockWidth):
        for starty in range(blockHeight):
            pixelBlocks.append(makeBlock(starty*7,startx*7,testImage))

    if(prt==True):
        print("實際總區塊數量: "+str(len(pixelBlocks)))

    #像素轉二維陣列
    binaryPixelBlocks=[]
    for pixelBlock in pixelBlocks:
        binaryPixelBlocks.append(turnBlocksToBinary(pixelBlock))
    
    if(prt==True):
        print("二維區塊數: "+str(len(binaryPixelBlocks)))

    #取得LSB區塊
    LSBBlocks=[]
    for binaryBlock in binaryPixelBlocks:
        LSBBlocks.append(turnBinaryBlockToLSBBlock(binaryBlock))

    if(prt==True):  
        print("LSB區塊數: "+str(len(LSBBlocks)))
    
    # 奇同位修改
    for LSBBlock in LSBBlocks:
        coverBlock(LSBBlock)
    
    if(prt==True):  
        print("奇同位修改完成")

    # 嵌入訊息
    for index,LSBBlock in enumerate(LSBBlocks):
        embeddLSBBlock(LSBBlock,getACharMessage(message,index))
    if(prt==True):  
        print("嵌入訊息完成")

    #LSB復原到像素區塊
    for index in range(blocksSize):
        reviseBinaryBlock(index,binaryPixelBlocks,LSBBlocks)#LSB復原到二進制區塊
        revisePixelBlock(index,binaryPixelBlocks,pixelBlocks)#二進制復原到像素區塊
    if(prt==True):  
        print("LSB復原到像素區塊完成")

    # 建立空影像
    hidedImage=[]
    for index in range(224):
        row=[225 for i in range(224)]
        hidedImage.append(row)
    for index,block in enumerate(pixelBlocks):
        putPixelBlockBack(int(index/32),int(index%32),block,hidedImage)
    if(prt==True):  
        print("隱藏圖像像素陣列建立完成")

    storeImage(hidedImage,path)
    if(prt==True):  
        print("隱藏圖像儲存完成，路徑為"+str(path))

    # 計算 psnr
    hidedImage = cv2.imread(path)
    hidedImage = cv2.cvtColor(hidedImage,cv2.COLOR_BGR2GRAY) 
    if(prt==True):  
        print('psnr= '+str(psnr(testImage,hidedImage))) #57

    # 計算 相對熵(Relative Entropy)
    x = np.reshape(hidedImage, -1)
    y = np.reshape(testImage, -1)
    
    if(prt==True):  
        print('相對熵= '+str(hxx_forward(x, y)))
    
    return psnr(testImage,hidedImage),hxx_forward(x, y)
        
    

In [35]:
PRDHHCSingle(testImage,message,True,'../database/PRDHHC/test.jpg')

原圖像素: (224, 224)
區塊數量: 32 * 32
總區塊數量: 1024
實際總區塊數量: 1024
二維區塊數: 1024
LSB區塊數: 1024
奇同位修改完成
嵌入訊息完成
LSB復原到像素區塊完成
隱藏圖像像素陣列建立完成
隱藏圖像儲存完成，路徑為../database/PRDHHC/test.jpg
psnr= 51.551846388872164
相對熵= 4.125020806553498


(51.551846388872164, 4.125020806553498)

In [36]:
PRDHHCSingle(testImage,message,False,'../database/PRDHHC/test.jpg')

(51.59775532397373, 4.119600880893148)

### 多個影像生成

#### 輸入

 message: 隱藏訊息

 prt: 是否顯示提示
 
 dirPath: 隱藏圖像儲存路徑

In [30]:
def getResizeImageArray():
    imageResizeArray=[]
    imageArray,imageArrayLength=getImageArray('../database/resizeImage')
    for image in imageArray:
        imageResizeArray.append(cv2.cvtColor(image,cv2.COLOR_BGR2GRAY))
    return imageResizeArray,imageArrayLength

In [31]:
def writeCSV(array,fileName):
    with open(fileName, 'w', newline='') as csvfile:
        # 以空白分隔欄位，建立 CSV 檔寫入器
        writer = csv.writer(csvfile, delimiter=',')

        writer.writerow(['PRDHHC'])
        for number in array:
            writer.writerow([number])
    csvfile.close()

In [32]:
def PRDHHCMultiple(message,prt,dirPath):
    psnrArray=[]
    hxxArray=[]
    #取得圖片陣列
    imageArray,imageArrayLength=getResizeImageArray()
    for index in range(imageArrayLength):
        psnr,hxx=PRDHHCSingle(imageArray[index],message,prt,dirPath+str(index+1)+'.jpg')
        psnrArray.append(psnr)
        hxxArray.append(hxx)
    
    if(prt):
        print(psnrArray)
        print(hxxArray)
    
    writeCSV(psnrArray,'./PSNR.csv')
    writeCSV(hxxArray,'./hxx.csv')


In [33]:
PRDHHCMultiple(message,False,'./database/')